In [355]:
url_prefix = "https://en.wikipedia.org/wiki/"
url_suffix = "_UEFA_Champions_League"
years_range = range(1992 , 2023)
url_array = []

for i in range(len(years_range) - 1):
    url_array.append(url_prefix + str(years_range[i]) + "%E2%80%93" + str(years_range[i+1])[2:] + url_suffix)

url_array

['https://en.wikipedia.org/wiki/1992%E2%80%9393_UEFA_Champions_League',
 'https://en.wikipedia.org/wiki/1993%E2%80%9394_UEFA_Champions_League',
 'https://en.wikipedia.org/wiki/1994%E2%80%9395_UEFA_Champions_League',
 'https://en.wikipedia.org/wiki/1995%E2%80%9396_UEFA_Champions_League',
 'https://en.wikipedia.org/wiki/1996%E2%80%9397_UEFA_Champions_League',
 'https://en.wikipedia.org/wiki/1997%E2%80%9398_UEFA_Champions_League',
 'https://en.wikipedia.org/wiki/1998%E2%80%9399_UEFA_Champions_League',
 'https://en.wikipedia.org/wiki/1999%E2%80%9300_UEFA_Champions_League',
 'https://en.wikipedia.org/wiki/2000%E2%80%9301_UEFA_Champions_League',
 'https://en.wikipedia.org/wiki/2001%E2%80%9302_UEFA_Champions_League',
 'https://en.wikipedia.org/wiki/2002%E2%80%9303_UEFA_Champions_League',
 'https://en.wikipedia.org/wiki/2003%E2%80%9304_UEFA_Champions_League',
 'https://en.wikipedia.org/wiki/2004%E2%80%9305_UEFA_Champions_League',
 'https://en.wikipedia.org/wiki/2005%E2%80%9306_UEFA_Champions_L

In [356]:
# https://stackoverflow.com/questions/3376803/beautifulsoup-find-th-with-text-price-then-get-price-from-next-th
# https://stackoverflow.com/questions/68612714/web-scraping-all-text-between-tabletable-i-need-table-in-python
# https://www.geeksforgeeks.org/how-to-find-a-html-tag-that-contains-certain-text-using-beautifulsoup/
# https://stackoverflow.com/questions/24748445/beautiful-soup-using-regex-to-find-tags
# https://www.geeksforgeeks.org/remove-all-style-scripts-and-html-tags-using-beautifulsoup/

import re
import pandas as pd
import requests
from bs4 import BeautifulSoup, NavigableString, Tag

columns=["Pos" , "Team" , "Pld" , "W" , "D" , "L" , "GF" , "GA" , "GD" , "Pts"]

group_stage_92_93 = BeautifulSoup(requests.get( url_array[12] ).content, "html.parser")

In [357]:
group_A_all_info = group_stage_92_93.find_all(lambda tag: tag.name == "th" and "Pld" in tag.text)[4].parent.parent

In [358]:
for abbr_title in group_A_all_info.find_all('abbr'):
    abbr_title.parent.decompose()

for style_tag in group_A_all_info(['style']):
    style_tag.decompose()
    
group_A_all_info.find_all(lambda tag:tag.name=="th" and "Team" in tag.text)[0].parent.decompose()

In [359]:
for qualification_th in group_A_all_info.find_all(lambda tag: tag.name == "th" and "Qualification" in tag.text):
    qualification_th.decompose()
    
for advance_to_td in group_A_all_info.find_all(lambda tag: tag.name == "td" and "Advance to" in tag.text):
    advance_to_td.decompose()
    
for transfer_to_td in group_A_all_info.find_all(lambda tag: tag.name == "td" and "Transfer to" in tag.text):
    transfer_to_td.decompose()

In [360]:
for head_to_head_td in group_A_all_info.find_all(lambda tag: tag.name == "td" and "—" in tag.text):
    head_to_head_td.decompose()
    
for head_to_head_td in group_A_all_info.find_all('a', {'href': re.compile('_v_')}):
    head_to_head_td.parent.decompose()
    
for empty_tds in group_A_all_info.find_all("td"):
    if empty_tds.text.strip() == "":
        empty_tds.decompose()

In [361]:
for abbr in group_A_all_info.find_all('th', {'scope': "col"}):
    if len(abbr.find_all("span")) >0:
        abbr.decompose()
        
for empty_ths in group_A_all_info.find_all("th"):
    if empty_ths.text.strip() == "":
        empty_ths.decompose()

In [362]:
all_data = []

for tr in group_A_all_info:
    if isinstance(tr, NavigableString):
        continue
    if isinstance(tr, Tag):
        th = tr.find("th")
        th.name = "td"
        tds = [td.get_text(strip=True) for td in tr.select("td")]
        all_data.append(tds)
    
df = pd.DataFrame(all_data,columns=columns,)

df

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Arsenal,6,2,4,0,11,6,+5,10
1,2,PSV Eindhoven,6,3,1,2,6,7,−1,10
2,3,Panathinaikos,6,2,3,1,11,8,+3,9
3,4,Rosenborg,6,0,2,4,6,13,−7,2
